In [1]:
from lib import *
import time
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
def flatten(b):
    m = len(b)
    n_x,n_y = b.shape
    b_flatten = np.zeros(n_x*n_y)
    for i in range(n_x):
        for j in range(n_y):
            index = linearize(i,j,m)

            b_flatten[index] =b[i,j]

            
    return b_flatten

In [3]:
def multigrid_iter2(b, h = None,v0=1, v1 = 2, v2=2, fullweight=True, plot=False, ax=None, fig=None):
    m = len(b)
    if not h:
        h = 1./(m-1)
    
    if (m-1)/2+1 == calculate_coarser(m):
        A = create_system(m,h=h)
        shapeF = b.shape
        b = flatten(b)
        return np.linalg.solve(A,b).reshape(shapeF), 0
        
        
    else:
        bC = restrict(b, fullweight=fullweight)
        xC,opsC = multigrid_iter2(bC,h=2*h,v0=v0,v1=v1,v2=v2,fullweight=fullweight,plot=plot,ax=ax,fig=fig)
        
        
        x = interpolate(xC)
        for i in range(v0):
            x,ops = vcycle_iteration(x, b, h=h , v1=v1,v2=v2,fullweight=fullweight,plot=plot,ax=ax,fig=fig)
        return x,ops+opsC/4.

In [4]:
def full_multigrid2(m,v0=1,v1=2,v2=2, plot=False,rest_type=None, atol=1e-3, random_seed= -1, demo=False):
    if m%2==0:
        raise Exception("This implementation works only with odd grids")
    
    if rest_type:
        if rest_type == "injection":
            fullweight = False
        elif rest_type =="full_weight":
            fullweight = True
        else:
            raise Exception("Restriction not supported")
    else:
        fullweight = True
    
    if plot:
        fig,ax = plt.subplots(1,1)
    else:
        fig,ax = (None,None)
    
    coarser = calculate_coarser(m)
    
    if coarser > 10:
        raise Exception("Coarser grid too big for direct solver {}x{}".format(coarser,coarser))

    h = 1./(m-1)
    
    x = np.linspace(0,1,m+2)[1:-1]
    y = np.linspace(0,1,m+2)[1:-1]    
    
    f = create_right(x,y)
    
    if random_seed > 0:
        np.random.seed(random_seed)
        v = np.random.rand(m,m)
    else:
        v = np.zeros((m,m))
 
    r = np.linalg.norm(calculate_residue(v,f,h))
    ops_total = 0
    v,ops = multigrid_iter2(f,h = h, v0 = v0, v1=v1, v2=v2, fullweight=fullweight, plot=plot, ax=ax, fig=fig)
    r = np.linalg.norm(calculate_residue(v,f,h))
    ops_total += ops

    return v,ops_total

In [5]:
nodes = 65
u = create_solution(nodes)
x = np.linspace(0,1,nodes+2)[1:-1]
f = create_right(x,x)
h = 1./(nodes-1)
b_residue = 1e10
b_error = 1e10

for i in range(1,300):
    for j in range(1,11):
        for k in range(1,11):
            v,ops = full_multigrid2(65,v0=i,v1=j,v2=k,rest_type="injection")
            
            error = np.linalg.norm(u-v)
            residue =np.linalg.norm(calculate_residue(v,f,h=h))
            
            if error < b_error:
                b_error = error
                b_error_v = (v,i,j,k)
            
            if residue < b_residue:
                b_residue = residue
                b_residue_v = (v,i,j,k)
            

KeyboardInterrupt: 

In [ ]:
full_multigrid2(65)